<img width="50px" align="left" style="margin-right:20px" src="./extra/images/npl_logo.png"> <b>New Professions Lab</b> <br /> Специалист по большим данным

# Проект 2

# Построить content-based рекомендательную систему товаров интернет-магазина ozon.ru

<img width="200px" align="left" src="./extra/images/ozon.png">

<a href="https://github.com/newprolab/content_bigdata9"><img align="left" src="./extra/images/npl7.svg"></a>

### Задача

Нужно построить рекомендательный алгоритм наилучшей предсказательной точности по метрике NDCG@100.

<!-- Подробнее о метрике: https://www.kaggle.com/wiki/NormalizedDiscountedCumulativeGain -->

Можно прочесть качественный пост на FastML: http://fastml.com/evaluating-recommender-systems/

Или на русском https://habr.com/company/econtenta/blog/303458/

Как и в первом проекте, мы рекомендуем сделать всё решение в этом ноутбуке. Но, в отличие от первого проекта, где всё-таки были очерчены примерные способы решения, в данном случае вам предоставляется полная свобода для трактования, исследования и выбора подходов к задаче. Более того, нет гарантий, что задача вообще решаема с необходимой точностью.

⏰ **Дедлайн: Воскресенье, 23 июня 2019, 23:59 по Москве**

### Обработка данных на вход

Для выполнения работы вам следует взять данные из HDFS:

```bash
$ hadoop fs -ls /labs/project02 
Found 6 items
-rw-r--r--   3 hdfs hdfs   13550896 2017-04-22 15:04 /labs/project02/catalog_path
-rw-r--r--   3 hdfs hdfs  453269519 2017-04-22 15:04 /labs/project02/catalogs
-rw-r--r--   3 hdfs hdfs 8726734647 2017-04-22 15:05 /labs/project02/item_details_full
-rw-r--r--   3 hdfs hdfs    2240879 2017-04-22 15:05 /labs/project02/ozon_test.txt
-rw-r--r--   3 hdfs hdfs   19097164 2017-04-22 15:05 /labs/project02/ozon_train.txt
-rw-r--r--   3 hdfs hdfs   12906349 2017-04-22 15:05 /labs/project02/ratings
```

Итак, давайте разберёмся, какие входные данные мы имеем:

#### `ozon_train.txt`

Обучающая выборка строчки в json, где для товара `item` мы предоставляем наиболее популярные рекомендации в `true_recoms` (здесь словарь из id рекомендуемого товара и веса — чем больше, тем лучше). Веса означают клики. Текущая рекомендательная система Ozon.ru — это смесь content-based и коллаборативной фильтрации.

Пример:

```
{"item":"24798277","true_recoms":{"24798314":1,"24798279":2,"24798276":4,"24798277":1,"24798280":2}}
```

❗️ В файле есть строчка с 40 000 рекомендаций — это мусор.

#### `ozon_test.txt`

Тестовая выборка.

Пример:

```
{"item": "28759795","recoms": null}
```

#### `item_details_full`

Атрибуты товаров.

Пример:

```
{"id":"4381194","name":"Графиня де Монсоро - В двух томах - Номерованный экземпляр № 84 (подарочное издание)","annotation":"Настоящее издание отпечатано в количестве тысячи пятисот экземпляров, сто из которых изготовлены в переплетах из черной кожи с золотыми обрезами и пронумерованы.  Номер настоящего экземпляра 84.<br>\r\n\"Графиня де Монсоро\" (1846) - одно из самых значительных произведений Александра Дюма. В этом увлекательном авантюрно-историческом романе писатель с замечательным мастерством воскрешает события второй половины XVI века - эпохи религиозных войн и правления Генриха III, последнего короля династии Валуа. История трагической любви благородного графа де Бюсси и прекрасной Дианы де Монсоро развертывается на фоне придворных интриг, политических заговоров и религиозных раздоров. <br>\r\nВ настоящем издании впервые публикуются все 245 иллюстраций выдающегося французского художника Мориса Лелуара, выполненные им для парижского издания 1903 года. Книга дополнена очерком А. И. Куприна \"Дюма-отец\" и обстоятельными комментариями.","parent_id":"18255189"}
```

❗️Под `parent_id` объединяются модификации одного товара (например, разные айфоны).


#### `catalogs`

В каких каталогах лежит товар (может быть несколько записей). 

Пример:

```
{"itemid":"29040016","catalogid":"1179259"}
```

#### `catalog_path`

Пути для каталогов нижнего уровня (в которых лежат товары) в дереве каталогов. Для каждого каталога отдаётся полный путь до корня. 

Пример:

```
{"catalogid":1125630,"catalogpath":[{"1125630":"Изысканные напитки. Сигары"},{"1125623":"Книга - лучший подарок!"},{"1112250":"Архив раздела (Нехудож.лит-ра)"},{"1095865":"Нехудожественная литература"}]}
```

#### `ratings`

Средний рейтинг `itemid` (звёздочки). 

Пример:

```
{“itemid”: 2658646, “rating”:4.0}
```

### Обработка данных на выход

Выходной файл должен иметь следующий формат (пример для одной строчки). Вес товара тем выше, чем выше его близость:

```
{"item": "28759795", "recoms": {"28759801": 1, "28759817": 2, "28759803": 13}}
```

Вы можете использовать любые алгоритмы и их смеси для предсказания рейтингов. Мы будем оценивать точность работы вашего алгоритма, рассчитывая средний NDCG@100 по всем товарам. Это означает, что для каждого `item` в тестовой выборке вы рекомендуете 100 товаров.

Чекер выглядит следующим образом:

```python
lines_number_ok = False
score = 0.0

file_exists = False
test_passed = False
true_recs = {}
lines_number = 0
lines_number_hidden = 0


try:
    if not cli.startcheck():
        exit(-1)

    file_exists = cli.checkfileexists(filename)

    if file_exists:
        with open(cli.getans('test_file_path')) as f:
            for line in f:
                data = json.loads(line)
                true_recs[data['item']] = data['true_recoms']
        lines_number_hidden = len(true_recs.keys())
        
        ndcg_sum = 0
        with open(cli.getfilepath(filename)) as f:
            for line in f:
                lines_number += 1
        
        if lines_number == len(true_recs.keys()):
            lines_number_ok = True
            with open(cli.getfilepath(filename)) as f:
                for line in f:                     
                    data = json.loads(line)     
                    trs = true_recs[data['item']]
                    tmrs = data['recoms']    

                    sorted_trs = sorted(trs.items(), key = lambda x: float(x[1]), reverse = True)
                    sorted_tmrs = sorted(tmrs.items(), key = lambda x: float(x[1]), reverse = True)
                    dcg = 0
                    idcg = 0

                    for i in range(len(trs)):
                        delta = sorted_trs[i][1]
                        if i + 1 != 1:
                            delta = delta / float(math.log(i + 1, 2))
                        idcg += delta

                    for i in range(len(tmrs)):
                        if sorted_tmrs[i][0] in trs:                    
                            delta = trs[sorted_tmrs[i][0]]
                            if i + 1 != 1:
                                delta = delta / float(math.log(i + 1, 2))
                            dcg += delta

                    ndcg_sum += dcg / idcg
            score = ndcg_sum / lines_number

            if score > 0.1:
                test_passed = True

```

Ваши результаты будут заноситься на обновляемую доску лидеров на [странице Проекта 2](http://lk.newprolab.com/lab/project02).

### Проверка

Для автоматической проверки необходимо сохранить заполненный предсказанными вами рейтингами файл `ozon_test.txt` в вашей домашней директории под именем `project02.txt`.

**ВАЖНО: Для точной проверки сохраняйте порядок и количество строк исходного файла.**

<p style="color:DarkRed"><b>✅ Проект будет засчитан, если вы преодолеете порог 0.1.</b></p>

### Подсказки

1. Обратите внимание на размер датасета.

2. Не забывайте мониторить свои ресурсы: если выполняете long-running jobs, сохраняйте результаты в конце и освобождайте память. Помните, что простаивающие процессы, держащие в памяти гигабайты данных, могут отстреливаться.

3. “Когда в руках молоток, все начинает напоминать гвоздь”. Думайте outside of the box.


<img width="60px" align="left" style="margin-right:20px" src="./extra/images/npl_logo.png"> <br /><b>Желаем вам удачи и успехов!</b>

## Ваше решение здесь

In [44]:
# Запуск pyspark
import os
import sys
os.environ["PYSPARK_SUBMIT_ARGS"]='pyspark-shell'
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')
sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

Welcome to
      ____              __
     / __/__  ___ _____/ /__
    _\ \/ _ \/ _ `/ __/  '_/
   /__ / .__/\_,_/_/ /_/\_\   version 2.4.3
      /_/

Using Python version 3.6.5 (default, Apr 29 2018 16:14:56)
SparkSession available as 'spark'.


In [45]:
sc.setCheckpointDir('checkpoint/')

In [46]:
from pyspark.sql.functions import col, explode, size, array, sort_array, collect_list, lit, desc, \
                                    udf, map_keys, map_values, concat, concat_ws
from pyspark.sql.types import *

In [47]:
import pandas as pd
from pandas.io.json import json_normalize
import json
import heapq

# EDA

### *Load and review file ozon_train.txt*

In [48]:
schema = StructType(fields=[StructField("item", StringType()),
                            StructField("true_recoms", MapType(StringType(), IntegerType()))])

train_data = spark.read.json("/labs/project02/ozon_train.txt", schema=schema)
%time train_data.show(5, truncate=False)

+--------+------------------------------------------------------------------------------------------------------+
|item    |true_recoms                                                                                           |
+--------+------------------------------------------------------------------------------------------------------+
|31471303|[31471373 -> 1, 31471307 -> 3, 31471342 -> 1]                                                         |
|31489016|[31760003 -> 1, 31418187 -> 1, 31418153 -> 1, 31740094 -> 1, 31489223 -> 1, 31489022 -> 1]            |
|5629613 |[19063915 -> 6, 5821229 -> 1, 19063895 -> 5, 19063889 -> 14, 5629614 -> 6, 5629622 -> 2, 7279030 -> 1]|
|24829740|[32610140 -> 1, 24829738 -> 1, 24702299 -> 1]                                                         |
|29185962|[1493930 -> 1, 3711112 -> 1, 33395696 -> 1]                                                           |
+--------+------------------------------------------------------------------------------

In [49]:
# Add column with number of true_recoms
train_data = train_data.withColumn('size_recoms', size('true_recoms')) \
                       .withColumnRenamed('item', 'train_item')
%time train_data.show(5)

+----------+--------------------+-----------+
|train_item|         true_recoms|size_recoms|
+----------+--------------------+-----------+
|  31471303|[31471373 -> 1, 3...|          3|
|  31489016|[31760003 -> 1, 3...|          6|
|   5629613|[19063915 -> 6, 5...|          7|
|  24829740|[32610140 -> 1, 2...|          3|
|  29185962|[1493930 -> 1, 37...|          3|
+----------+--------------------+-----------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 202 ms


In [5]:
%time train_data.summary().show()

+-------+--------------------+------------------+
|summary|          train_item|       size_recoms|
+-------+--------------------+------------------+
|  count|              142438|            142438|
|   mean|2.2748455783286646E7| 6.450610090004072|
| stddev|1.0655741450096913E7|119.61303591982087|
|    min|                    |                 3|
|    25%|         1.7953518E7|                 3|
|    50%|         2.7582691E7|                 5|
|    75%|          3.159911E7|                 7|
|    max|              997604|             45111|
+-------+--------------------+------------------+

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 11.2 s


#### удалим "мусорную" строку

In [7]:
# найдем мусорную строку с >40 тыс рекомендаций
%time train_data.filter(size("true_recoms")>40000).show()

+----------+--------------------+-----------+
|train_item|         true_recoms|size_recoms|
+----------+--------------------+-----------+
|          |[33192962 -> 1, 2...|      45111|
+----------+--------------------+-----------+

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 8.68 s


In [50]:
# удалим эту мусорную строку без item с 45 тыс рекомендаций
train_data = train_data.filter(size("true_recoms")<40000)
%time train_data.summary().show()

+-------+--------------------+-----------------+
|summary|          train_item|      size_recoms|
+-------+--------------------+-----------------+
|  count|              142437|           142437|
|   mean|2.2748455783286646E7|6.133946937944494|
| stddev|1.0655741450096913E7|4.922344408445034|
|    min|              100304|                3|
|    25%|         1.7953518E7|                3|
|    50%|         2.7582691E7|                5|
|    75%|          3.159911E7|                7|
|    max|              997604|              158|
+-------+--------------------+-----------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 10.2 s


### *Load and review file ozon_test.txt*

In [51]:
schema = StructType(fields=[StructField("item", StringType()),
                            StructField("recoms", StringType())])

test_data = spark.read.json("/labs/project02/ozon_test.txt", schema=schema)
%time test_data.show(5)

+--------+------+
|    item|recoms|
+--------+------+
|28759795|  null|
|32712593|  null|
|29616882|  null|
| 6242189|  null|
|32663967|  null|
+--------+------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 214 ms


In [14]:
# How many items we need to make prediction for?
%time test_data.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 745 ms


60956

### *Load and review file with catalogs*

In [52]:
schema = StructType(fields=[StructField("itemid", StringType()),
                            StructField("catalogid", StringType())])

catalog_data = spark.read.json("/labs/project02/catalogs", schema=schema)
%time catalog_data.show(5)

+--------+---------+
|  itemid|catalogid|
+--------+---------+
|26881367|  1139010|
|31587713|  1140991|
|17425282|  1134295|
| 6907117|  1138615|
| 7868912|  1144454|
+--------+---------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 115 ms


In [16]:
# Number of items in catalog_data
%time catalog_data.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.4 s


10382286

In [13]:
# Number of distict itemid in catalog_data
%time catalog_data.select('itemid').distinct().count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 13.9 s


6732538

In [14]:
# Number of distinct catalogid in catalog_data
%time catalog_data.select('catalogid').distinct().count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 5.62 s


4780

### *Load and review file with catalog_paths*

In [53]:
schema = StructType(fields=[StructField("catalogid", StringType()),
                            StructField("catalogpath", ArrayType(MapType(StringType(), StringType())))])

path_data = spark.read.json("/labs/project02/catalog_path", schema=schema)
%time path_data.show(5, truncate=False)

+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|catalogid|catalogpath                                                                                                                                                                                                            |
+---------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|1125369  |[[1125369 -> Аксессуары], [1123855 -> Коты], [1123646 -> Я_Символ года 2011], [1110181 -> Я_ Новогодняя тематика], [1106018 -> ПОДАРКИ. СЕЗОННЫЕ ПРЕДЛОЖЕНИЯ], [1134031 -> ПОДАРКИ_СТАРЫЕ ВЕТКИ_НЕ АКТИВНЫ]]           |
|1125370  |[[1125370 -> Магнитики], [1123855 -> Коты], [1123646 -> Я_Символ года 2011], 

In [17]:
# Number of items in path_data
%time path_data.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.48 s


54504

In [20]:
%time path_data.withColumn('path_size', size('catalogpath')).summary().show()

+-------+------------------+------------------+
|summary|         catalogid|         path_size|
+-------+------------------+------------------+
|  count|             54504|             54504|
|   mean|1102176.2992807867| 4.167125348598268|
| stddev|172767.14832080383|1.3473399496432081|
|    min|           1000000|                 1|
|    25%|         1097883.0|                 3|
|    50%|         1129049.0|                 4|
|    75%|         1158422.0|                 5|
|    max|              9997|                10|
+-------+------------------+------------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.76 s


In [54]:
# Extract path_id and text from path_data
path = path_data.withColumn('path_item0', col('catalogpath').getItem(0)) \
               .withColumn('path_item1', col('catalogpath').getItem(1)) \
               .withColumn('path_item2', col('catalogpath').getItem(2)) \
               .withColumn('path_item3', col('catalogpath').getItem(3)) \
               .withColumn('path0_txt', map_values('path_item0').getItem(0)) \
               .withColumn('path1_id', map_keys('path_item1').getItem(0)) \
               .withColumn('path1_txt', map_values('path_item1').getItem(0)) \
               .withColumn('path2_id', map_keys('path_item2').getItem(0)) \
               .withColumn('path2_txt', map_values('path_item2').getItem(0)) \
               .withColumn('path3_id', map_keys('path_item3').getItem(0)) \
               .withColumn('path3_txt', map_values('path_item3').getItem(0)) \
               .select('catalogid', 'path1_id', 'path2_id', 'path3_id', \
                       concat_ws(' ', 'path0_txt', 'path1_txt', 'path2_txt', 'path3_txt').alias('path_txt'))
%time path.show(5, truncate=False)

+---------+--------+--------+--------+--------------------------------------------------------------------+
|catalogid|path1_id|path2_id|path3_id|path_txt                                                            |
+---------+--------+--------+--------+--------------------------------------------------------------------+
|1125369  |1123855 |1123646 |1110181 |Аксессуары Коты Я_Символ года 2011 Я_ Новогодняя тематика           |
|1125370  |1123855 |1123646 |1110181 |Магнитики Коты Я_Символ года 2011 Я_ Новогодняя тематика            |
|1125372  |1123855 |1123646 |1110181 |Подсвечники и копилки Коты Я_Символ года 2011 Я_ Новогодняя тематика|
|1125373  |1123855 |1123646 |1110181 |Коты-игрушки Коты Я_Символ года 2011 Я_ Новогодняя тематика         |
|1125374  |1123855 |1123646 |1110181 |Сувениры Коты Я_Символ года 2011 Я_ Новогодняя тематика             |
+---------+--------+--------+--------+--------------------------------------------------------------------+
only showing top 5 rows

CPU

### *Load and review file with ratings*

In [55]:
schema = StructType(fields=[StructField("itemid", StringType()),
                            StructField("rating", DoubleType())])

rating_data = spark.read.json("/labs/project02/ratings", schema=schema)
rating_data.show(5)

+--------+------+
|  itemid|rating|
+--------+------+
| 2515646|   5.0|
| 2858646|   5.0|
| 5168646|   3.5|
| 6465646|   5.0|
|26351646|   5.0|
+--------+------+
only showing top 5 rows



In [31]:
rating_data.summary().show()

+-------+-----------------+------------------+
|summary|           itemid|            rating|
+-------+-----------------+------------------+
|  count|           384825|            384825|
|   mean|9376621.363217046| 4.260485264332202|
| stddev|9530095.465377059|1.0581581607083608|
|    min|          1000023|               1.0|
|    25%|        2925920.0|               4.0|
|    50%|        4820610.0|               5.0|
|    75%|      1.8442589E7|               5.0|
|    max|            99991|               5.0|
+-------+-----------------+------------------+



### *Load and review file item_details_full*

In [21]:
# Load the largest dataset 8Gb with items details
%time details_data = spark.read.json("/labs/project02/item_details_full") \
                          .withColumnRenamed('attr0', 'annotation') \
                          .withColumnRenamed('attr1', 'name') \
                          .withColumnRenamed('attr2', 'author')
details_data.show(1)

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 15.6 s
+--------------------+--------------------+------+------+------+------+------+------+------+------+--------------------+------+-----------+------+------+------+------+------+------+------+------+------+------+--------------------+------+------+---------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+------+------+-----+------+------+------+------+------+------+------+------+-----+-----+--------+---------+
|          annotation|                name|attr10|attr11|attr12|attr13|attr14|attr15|attr16|attr17|              attr18|attr19|     author|attr20|attr21|attr22|attr23|attr24|attr25|attr26|attr27|attr28|attr29|               attr3|attr30|attr31|   attr32|attr33|attr34|attr35|attr36|attr37|attr38|attr39|attr4|att

In [22]:
details_data.select('itemid', 'parent_id', 'name', 'annotation', 'author') \
            .show(1, vertical=True, truncate=False)

-RECORD 0----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 itemid     | 12808031                                                                                                                                                                                                                                                                                        

In [23]:
details_data.select('itemid', 'parent_id', 'name', 'annotation', 'author').show(10)

+--------+---------+--------------------+--------------------+-------------------+
|  itemid|parent_id|                name|          annotation|             author|
+--------+---------+--------------------+--------------------+-------------------+
|12808031| 17969540|История маленьког...|<B> Прижизненное ...|        Ф. Бернет  |
|12599476| 16095306|Belkin чехол для ...|Надежная защита -...|               null|
|22440986|     null|Толковый словарь ...|               нет  |     С. И. Ожегов  |
|22441264|     null|Архитектурное тво...|               нет  |               null|
|22466267| 22476179|Видео-открытка "Т...|Дорогие друзья!<b...|               null|
|22500510| 22611447|Ботинки West Club...|Удобные ботинки W...|               null|
|22501357| 22517964|Ботинки Betsy 329...|Отличные ботинки ...|               null|
|22437755|     null|Побег в соловьины...|Предлагаем вашему...|Владимир Кобликов  |
|22449833| 23139002|Играем в кости. 1...|Весело, с азартом...|    Райнер Книзиа  |
|223

In [24]:
details_data.rdd.getNumPartitions()

66

In [25]:
details_data = details_data.coalesce(8).cache()

In [26]:
details_data.select('itemid', 'parent_id', 'name', 'annotation', 'author').printSchema()

root
 |-- itemid: string (nullable = true)
 |-- parent_id: string (nullable = true)
 |-- name: string (nullable = true)
 |-- annotation: string (nullable = true)
 |-- author: string (nullable = true)



In [27]:
%time details_data.select('itemid', 'parent_id', 'name', 'annotation', 'author').describe().show()

+-------+--------------------+--------------------+--------------------+--------------------+------------------+
|summary|              itemid|           parent_id|                name|          annotation|            author|
+-------+--------------------+--------------------+--------------------+--------------------+------------------+
|  count|             9273625|             3805884|             9273137|             8272266|           5296280|
|   mean|1.7692237242282603E7|2.2682176065096308E7|                 NaN|            Infinity| 774.9794520547945|
| stddev|1.0924191469207602E7|   8119906.385594677|                 NaN|                 NaN|1070.2348412388437|
|    min|              100000|             1000000|              10x10 |                    |                  |
|    max|              999999|              999998|™konomie Der Balk...|№ 30 Спальный меш...|          № 2143  |
+-------+--------------------+--------------------+--------------------+--------------------+---

In [28]:
# select details data that has intersection with train and test datasets
details_data_small = details_data.join(train_data.select(col('train_item').name('itemid')), on='itemid', how='left_semi') \
            .union(details_data.join(test_data.select(col('item').name('itemid')), on='itemid', how='left_semi')) \
            .dropDuplicates()
%time details_data_small.count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 31.5 s


204183

In [29]:
%time details_data_small.count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 1min 4s


204183

# Находим подходящие пары для test items и присваиваем им вес для ранжирования


Pairs | Weight
:---- | :----
Friends | Rating + alpha x Views
Common Friends | Rating + alpha/4 x Views
Common Catalog | Rating
Common Path 1/2/3 | Rating/2/4/8
Common Parent | Rating

__where alpha is a scaling factor of confidence in implicit rating__


In [56]:
alpha = 32

## Idea 1. Статистика совместных просмотров (find Friends)

In [57]:
# Explode train data to get pairs of items with joint views
pairs = train_data.select("train_item", explode("true_recoms").alias("recoms", "views"))
%time pairs.show(5)

+----------+--------+-----+
|train_item|  recoms|views|
+----------+--------+-----+
|  31471303|31471373|    1|
|  31471303|31471307|    3|
|  31471303|31471342|    1|
|  31489016|31760003|    1|
|  31489016|31418187|    1|
+----------+--------+-----+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 194 ms


In [24]:
%time pairs.summary().show()

+-------+--------------------+--------------------+-----------------+
|summary|          train_item|              recoms|            views|
+-------+--------------------+--------------------+-----------------+
|  count|              873701|              873701|           873701|
|   mean|2.3911691907111242E7|2.4031108647896707E7|4.005077251828715|
| stddev| 1.010491250777942E7| 1.000269246709353E7| 19.6416931018131|
|    min|              100304|            10016747|                1|
|    25%|         1.9578092E7|         1.9688909E7|                1|
|    50%|         2.8106575E7|         2.8312045E7|                1|
|    75%|         3.1706643E7|         3.1701051E7|                4|
|    max|              997604|               99845|            10820|
+-------+--------------------+--------------------+-----------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 14.5 s


In [26]:
# Review items with most joint views
%time pairs.orderBy('views', ascending=False).show(10)

+----------+--------+-----+
|train_item|  recoms|views|
+----------+--------+-----+
|  32570730|32570811|10820|
|  32570811|32570730| 8709|
|  28906831|31053293| 2660|
|  32570730|33007311| 2051|
|  32465635|32666051| 2018|
|  32570730|32871593| 1983|
|  28906831|33211682| 1756|
|  32570811|33007311| 1730|
|  24298034|32570730| 1720|
|  32570730|31653745| 1698|
+----------+--------+-----+
only showing top 10 rows

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 37.5 s


### Filter *test items* in recoms of train items (we look for Friends of test items only)

In [58]:
# Select pairs with test recoms => friends of test items
pairs_test_f = pairs.join(test_data.select(col('item').name('recoms')), on='recoms', how='left_semi') \
                  .select(col('recoms').name('test_item'), 'train_item', 'views')
%time pairs_test_f.show(5)

+---------+----------+-----+
|test_item|train_item|views|
+---------+----------+-----+
| 31760003|  31489016|    1|
| 31418187|  31489016|    1|
| 19063915|   5629613|    6|
|  5821229|   5629613|    1|
| 33395696|  29185962|    1|
+---------+----------+-----+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 566 ms


In [29]:
# Revies summary of pairs_test_f
%time pairs_test_f.summary().show()

+-------+------------------+--------------------+------------------+
|summary|         test_item|          train_item|             views|
+-------+------------------+--------------------+------------------+
|  count|            183544|              183544|            183544|
|   mean|2.48761968697533E7|2.4612840692803904E7| 4.539500065379419|
| stddev| 9422779.428920371|   9580928.216531042|14.874518162512967|
|    min|          10028901|              101089|                 1|
|    25%|        2.034623E7|          2.020266E7|                 1|
|    50%|       2.9033245E7|         2.8515991E7|                 1|
|    75%|       3.1813565E7|         3.1775711E7|                 4|
|    max|             99411|               99611|              2051|
+-------+------------------+--------------------+------------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 15.8 s


In [27]:
# Number of test items with some friends
%time pairs_test_f.select('test_item').distinct().count()

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 11.5 s


47932

In [30]:
# Check if we have any test items with num of friens >=100,
%time pairs_test_f.select('test_item', array('train_item', 'views').name('recoms')) \
                .groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100').count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 6.61 s


0

In [31]:
# There is no test items with num of friends >=100 right now, check the max number:
%time pairs_test_f.select('test_item', array('train_item', 'views').name('recoms')) \
                .groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).agg({'num_recoms':'max'}).show()

+---------------+
|max(num_recoms)|
+---------------+
|             51|
+---------------+

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 8.09 s


### Compute weights for Friends

In [59]:
pairs_test_f = pairs_test_f.join(rating_data.select(col('itemid').name('train_item'), 'rating'), \
                                 on='train_item', how='left_outer') \
                           .fillna(0, subset=['rating']) \
                           .withColumn('weight', col('rating') + alpha * col('views')) \
                           .withColumnRenamed('train_item', 'recoms')
%time pairs_test_f.show(5)

+--------+---------+-----+------+------+
|  recoms|test_item|views|rating|weight|
+--------+---------+-----+------+------+
|31489016| 31760003|    1|   0.0|  32.0|
|31489016| 31418187|    1|   0.0|  32.0|
| 5629613| 19063915|    6|   4.5| 196.5|
| 5629613|  5821229|    1|   4.5|  36.5|
|29185962| 33395696|    1|   0.0|  32.0|
+--------+---------+-----+------+------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.52 s


## Idea 1' - Find Friends of Friends

### Join test items with friends with train items - "friends of friends"

In [62]:
temp1 = pairs_test_f.select('test_item', col('recoms').name('train_item'), 'weight') \
                .join(train_data.select('train_item', 'true_recoms'), \
                      on='train_item', how='left_outer') \
                .select('test_item', 'train_item', 'weight', explode('true_recoms').name('recoms', 'views')) \
                .join(rating_data.select(col('itemid').name('recoms'), 'rating'), \
                      on='recoms', how='left_outer') \
                .fillna(0, subset=['rating'])
%time temp1.show(5)

+--------+---------+----------+------+-----+------+
|  recoms|test_item|train_item|weight|views|rating|
+--------+---------+----------+------+-----+------+
|31760003| 31760003|  31489016|  32.0|    1|   0.0|
|31418187| 31760003|  31489016|  32.0|    1|   0.0|
|31418153| 31760003|  31489016|  32.0|    1|   0.0|
|31740094| 31760003|  31489016|  32.0|    1|   0.0|
|31489223| 31760003|  31489016|  32.0|    1|   0.0|
+--------+---------+----------+------+-----+------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.98 s


In [63]:
%time temp1.count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 10.7 s


2092525

### Union friends and friends of friends into one table

In [64]:
pairs_test_f1 = temp1.select('test_item', col('train_item').name('recoms'), 'weight') \
                   .union(temp1.select('test_item', 'recoms', \
                                       (col('rating') + (alpha/4) * col('views')).name('weight'))) \
                   .dropDuplicates().filter('test_item != recoms')                
%time pairs_test_f1.show(5)

+---------+--------+------+
|test_item|  recoms|weight|
+---------+--------+------+
| 32937677|32937080| 384.0|
| 30914782|26418638|  36.0|
|  5804037| 4645025| 416.0|
| 22443211| 3768754| 165.0|
|  6281222| 4188479|  69.0|
+---------+--------+------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 19.8 s


In [65]:
pairs_test_f1.rdd.getNumPartitions()

200

In [66]:
pairs_test_f1 = pairs_test_f1.coalesce(8).cache()

In [67]:
%time pairs_test_f1.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 11.3 s


1967638

In [71]:
# Number of test items with number of pairs >=100
%time pairs_test_f1.groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100').count()

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 3.82 s


4265

In [72]:
# Number of distinct test items with pairs
%time pairs_test_f1.select('test_item').distinct().count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.15 s


47932

## Idea 0 (emergency cases): Popular recoms for ANY test item

In [30]:
# Select 100 most popular pairs (joint views) when we do not enough pairs (joint views) !!!!!!!!
popular = pairs.orderBy('views', ascending=False).limit(100)
%time popular.show(10)

+----------+--------+-----+
|train_item|  recoms|views|
+----------+--------+-----+
|  32570730|32570811|10820|
|  32570811|32570730| 8709|
|  28906831|31053293| 2660|
|  32570730|33007311| 2051|
|  32465635|32666051| 2018|
|  32570730|32871593| 1983|
|  28906831|33211682| 1756|
|  32570811|33007311| 1730|
|  24298034|32570730| 1720|
|  32570730|31653745| 1698|
+----------+--------+-----+
only showing top 10 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.27 s


In [68]:
# Select 100 most popular pairs (joint views) when we do not enough pairs (joint views) !!!!!!!!
popular_test = pairs_test_f.sort(desc('views')).limit(100)
%time popular_test.show(10)

+--------+---------+-----+-----------------+------------------+
|  recoms|test_item|views|           rating|            weight|
+--------+---------+-----+-----------------+------------------+
|32570730| 33007311| 2051|4.761904761904762| 65636.76190476191|
|28906831| 33211682| 1756|              3.5|           56195.5|
|32570811| 33007311| 1730|4.944444444444445|55364.944444444445|
|32570730| 33191212| 1141|4.761904761904762| 36516.76190476191|
|32570730| 32167261| 1024|4.761904761904762| 32772.76190476191|
|32570811| 32167261|  981|4.944444444444445|31396.944444444445|
|23809424|  5438279|  923|4.214285714285714|29540.214285714286|
|31653745| 33007311|  843|              4.5|           26980.5|
|32698107| 32698108|  840|              0.0|           26880.0|
|32598788| 33000972|  802|              4.5|           25668.5|
+--------+---------+-----+-----------------+------------------+
only showing top 10 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.19 s


In [69]:
# Make pairs with popular items for emergency cases
pairs_test_pop = test_data.select(col('item').name('test_item')) \
                        .crossJoin(popular_test.select('recoms', col('rating').name('weight'))) \
                        .dropDuplicates()
%time pairs_test_pop.show(5)

+---------+--------+-----------------+
|test_item|  recoms|           weight|
+---------+--------+-----------------+
| 32712593|31138940|4.962962962962963|
|  6242189|32570811|4.944444444444445|
| 27976812|33255573|              0.0|
| 32788806|32175016|             4.75|
|  7015126|23809424|4.214285714285714|
+---------+--------+-----------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 10.6 s


In [70]:
pairs_test_pop.rdd.getNumPartitions()

200

In [71]:
pairs_test_pop = pairs_test_pop.coalesce(8).cache()

In [23]:
# number of pairs = number of test items x 100
%time pairs_test_pop.count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 36.6 s


6095600

## Idea 2: Find pairs with common catalogs

### Join test data with catalogs

In [74]:
# Join test items with catalogid
pairs_test_c = test_data.select(col('item').name('test_item')) \
                        .join(catalog_data.select(col('itemid').name('test_item'), 'catalogid'), \
                              on='test_item', how='left_outer') \
                        .join(catalog_data.select(col('itemid').name('recoms'), 'catalogid'), \
                                 on='catalogid', how='left_outer') \
                            .dropDuplicates() \
                        .join(rating_data.select(col('itemid').name('recoms'), col('rating').name('weight')), \
                                on='recoms', how='left_outer') \
                        .filter('weight > 0') \
                        .withColumn('weight', col('weight').cast('float'))
%time pairs_test_c.show(5)

+--------+---------+---------+------+
|  recoms|catalogid|test_item|weight|
+--------+---------+---------+------+
|18568083|  1142513| 18382630|   2.0|
| 2720113|  1142513| 18382630|   5.0|
| 7427843|  1142513| 18382630|   5.0|
|17874310|  1142513| 18382630|   5.0|
|18382630|  1142513| 18382630|   5.0|
+--------+---------+---------+------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 14.5 s


In [149]:
# Remove pairs already in friends
pairs_test_c = pairs_test_c.join(pairs_test_f.select('test_item', 'recoms'), \
                                 on=['test_item', 'recoms'], how='left_anti')
%time pairs_test_c.show(5)

+---------+--------+---------+------+
|test_item|  recoms|catalogid|weight|
+---------+--------+---------+------+
|  1012719| 3788261|  1139400|   1.0|
|   102012| 1330933|  1141109|   3.0|
|   102012|17910479|  1141109|   5.0|
|   102012|19042093|  1141109|   2.0|
|   102012|19931478|  1141109|   5.0|
+---------+--------+---------+------+
only showing top 5 rows

CPU times: user 336 ms, sys: 104 ms, total: 440 ms
Wall time: 46min 49s


In [148]:
pairs_test_c.rdd.getNumPartitions()

200

In [55]:
%time pairs_test_c.count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 15.5 s


82194

In [54]:
# Number of pairs with same catalogid
%time pairs_test_c.count()

CPU times: user 92 ms, sys: 28 ms, total: 120 ms
Wall time: 15min 22s


1582735788

## Union Friends and Items with same catalogid

In [151]:
# lets add them to pairs_test
pairs_test_after_c = pairs_test_f1.union(pairs_test_c.select('test_item', 'recoms', 'weight')) \
                               .dropDuplicates()
%time pairs_test_after_c.show(5)

+---------+--------+------+
|test_item|  recoms|weight|
+---------+--------+------+
| 32571873|32805257|  32.0|
|  5524498| 3080602| 352.0|
| 19447077|19447078| 356.2|
|  1661774|31919544| 352.0|
| 30002018|30002011|  96.0|
+---------+--------+------+
only showing top 5 rows

CPU times: user 364 ms, sys: 148 ms, total: 512 ms
Wall time: 44min 12s


In [80]:
pairs_test_after_c.rdd.getNumPartitions()

200

In [153]:
pairs_test_after_c = pairs_test_after_c.coalesce(20).cache()

In [86]:
%time pairs_test_after_c.count()

CPU times: user 4 ms, sys: 4 ms, total: 8 ms
Wall time: 21.1 s


90203583

In [88]:
# How many test items with pairs do we now have?
%time pairs_test_after_c.filter('recoms is not null').select('test_item').distinct().count()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 2.82 s


60373

In [89]:
# Number of test items with number of pairs >=100 (take median to reduce batch)
%time pairs_test_after_c.groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100').count()

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 13.1 s


46878

## Make 1st batch of num recoms >100

In [154]:
# Save the first batch of test items with number of recoms >=100
batch1 = pairs_test_after_c.join(pairs_test_after_c.groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100').select('test_item'), \
                                 on='test_item', how='left_semi') \
                .select(col('test_item').name('item'), 'recoms', 'weight')
%time batch1.show(5)

+--------+--------+------+
|    item|  recoms|weight|
+--------+--------+------+
|13078902|24807514|   3.0|
|13078902| 5020621|   5.0|
|13078902|19729265|   5.0|
|13078902|22707017|   5.0|
|13078902| 8785535|  88.0|
+--------+--------+------+
only showing top 5 rows

CPU times: user 396 ms, sys: 72 ms, total: 468 ms
Wall time: 44min 52s


In [155]:
batch1.rdd.getNumPartitions()

200

In [92]:
batch1 = batch1.coalesce(20).cache()

In [93]:
%time batch1.groupBy('item').agg(collect_list('recoms').name('recoms')) \
            .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100') \
            .select('num_recoms').summary().show()

+-------+-----------------+
|summary|       num_recoms|
+-------+-----------------+
|  count|            46878|
|   mean|1909.982187806647|
| stddev|4376.774550200061|
|    min|              100|
|    25%|              240|
|    50%|              592|
|    75%|             1486|
|    max|            27332|
+-------+-----------------+

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 58.3 s


In [94]:
# Number of pairs in batch1 - better < 50m for memory reason
%time batch1.count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 647 ms


89536145

In [95]:
batch1.printSchema()

root
 |-- item: string (nullable = true)
 |-- recoms: string (nullable = true)
 |-- weight: double (nullable = true)



### Collect recoms into array

In [156]:
# Group predictions by item and create arrays (recoms, weight)
out1 = batch1.groupBy('item').agg(collect_list(array('recoms', 'weight')).alias('recoms'))
%time out1.show(5)

+--------+--------------------+
|    item|              recoms|
+--------+--------------------+
|13078902|[[19415756, 5.0],...|
| 1513794|[[29684447, 5.0],...|
|15637425|[[4993237, 5.0], ...|
| 1574856|[[19083208, 21.0]...|
|18574475|[[8265004, 5.0], ...|
+--------+--------------------+
only showing top 5 rows

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 20.2 s


In [85]:
# Note that collection to array changed type of weights to string
out1.printSchema()

root
 |-- item: string (nullable = true)
 |-- recoms: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: string (containsNull = true)



## Save batch1 to a Pandas dataframe

### Load test_data to Pandas

In [86]:
# Better load test data to Pandas directly to preserve order
df_test = pd.read_csv("data/ozon_test.txt", header=None, sep="\n")
df_test.head()

,0
0,"{""item"": ""28759795"", ""recoms"": null}"
1,"{""item"": ""32712593"", ""recoms"": null}"
2,"{""item"": ""29616882"", ""recoms"": null}"
3,"{""item"": ""6242189"", ""recoms"": null}"
4,"{""item"": ""32663967"", ""recoms"": null}"


In [87]:
# Normalize data to create a dataframe
test_data_pd = json_normalize(df_test.loc[:, 0].apply(json.loads))
test_data_pd.head()

,item
0,28759795
1,32712593
2,29616882
3,6242189
4,32663967


### Save batch1 to Pandas

In [157]:
%time res1 = out1.toPandas()
res1.head(2)

CPU times: user 42.4 s, sys: 18.6 s, total: 1min 1s
Wall time: 1min 33s


,item,recoms
0,13078902,"[[4411393, 4.5], [8777322, 4.695652008056641],..."
1,1513794,"[[2763322, 1.0], [32674999, 5.0], [5212243, 4...."


### Batch1 top100 recoms by weight

In [158]:
# Make dict from recoms, convert weights from string to float and select top100 by weight
%time res1['recoms'] = res1['recoms'].apply(dict) \
            .apply(lambda x: dict((k, float(v)) for k,v in x.items())) \
            .apply(lambda x: dict(heapq.nlargest(100, x.items(), key=lambda i: i[1])))

CPU times: user 3min 45s, sys: 40.2 s, total: 4min 25s
Wall time: 4min 24s


In [159]:
res1.head(2)

,item,recoms
0,13078902,"{'8785535': 88.0, '2692026': 32.0, '8748503': ..."
1,1513794,"{'32674999': 5.0, '3784919': 5.0, '19977275': ..."


### Add batch1 to test_data_pd

In [160]:
# Make sure we have a correct order of items
test_data_pd1 = test_data_pd.merge(res1, on='item', how='left')
test_data_pd1.head()

,item,recoms
0,28759795,NaN
1,32712593,NaN
2,29616882,"{'29596054': 227.0, '29596018': 221.0, '295960..."
3,6242189,NaN
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [161]:
test_data_pd1.recoms.count()

46878

## Remaining test items after removing batch1

In [96]:
pairs_test_rem1 = pairs_test_after_c.join(batch1.select(col('item').name('test_item')), on='test_item', how='left_anti')
%time pairs_test_rem1.show(5)

+---------+--------+------+
|test_item|  recoms|weight|
+---------+--------+------+
| 18555582| 7518812|   5.0|
| 18555582|18555590|   3.0|
| 18555582| 7518816|   5.0|
| 18555582| 3820346|   5.0|
| 18555582| 5624460|   5.0|
+---------+--------+------+
only showing top 5 rows

CPU times: user 8 ms, sys: 0 ns, total: 8 ms
Wall time: 26.6 s


In [97]:
pairs_test_rem1.rdd.getNumPartitions()

200

In [98]:
pairs_test_rem1 = pairs_test_rem1.coalesce(8).cache()

In [99]:
%time pairs_test_rem1.count()

CPU times: user 80 ms, sys: 312 ms, total: 392 ms
Wall time: 29.3 s


664584

In [100]:
%time pairs_test_rem1.select('test_item').distinct().count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.17 s


13434

### Collect remaining recoms (<100 per item) into array

In [162]:
# Group predictions by item and create arrays (recoms, weight)
out_rem1 = pairs_test_rem1.withColumnRenamed('test_item', 'item') \
                    .groupBy('item').agg(collect_list(array('recoms', 'weight')).alias('recoms'))
%time out_rem1.show(5)

+--------+--------------------+
|    item|              recoms|
+--------+--------------------+
|18555582|[[18555590, 3.0],...|
|19148459|[[32873012, 8.0],...|
|19148526|[[32021517, 8.0],...|
|19467872|[[7610827, 5.0], ...|
|19705179|[[20396906, 1.0],...|
+--------+--------------------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 12.8 s


### Save remaining recoms to Pandas

In [163]:
%time res_rem1 = out_rem1.toPandas()
res_rem1.head(2)

CPU times: user 496 ms, sys: 80 ms, total: 576 ms
Wall time: 1.15 s


,item,recoms
0,18555582,"[[18555590, 3.0], [7518816, 5.0], [7518819, 5...."
1,19148459,"[[32873012, 8.0], [2084565, 3.0], [31227861, 3..."


### Transform remaining recoms same way as batch1

In [164]:
# Make dict from recoms, convert weights from string to float and select top100 by weight
%time res_rem1['recoms'] = res_rem1['recoms'].apply(dict) \
            .apply(lambda x: dict((k, float(v)) for k,v in x.items())) \
            .apply(lambda x: dict(heapq.nlargest(100, x.items(), key=lambda i: i[1])))
res_rem1.head(2)

CPU times: user 1.57 s, sys: 284 ms, total: 1.85 s
Wall time: 1.85 s


,item,recoms
0,18555582,"{'7518816': 5.0, '7518819': 5.0, '18555586': 5..."
1,19148459,"{'24839584': 128.0, '31879593': 53.0, '3187959..."


### Add remaining records to test_data_pd for submission

In [165]:
# Make sure we have a correct order of items
test_data_rem1 = test_data_pd.merge(res_rem1, on='item', how='left')
test_data_rem1.head()

,item,recoms
0,28759795,"{'31413158': 96.0, '31594180': 88.0, '28759792..."
1,32712593,"{'32712245': 112.0, '32712635': 88.0, '3271264..."
2,29616882,NaN
3,6242189,NaN
4,32663967,NaN


In [166]:
test_data_rem1.recoms.count()

13434

In [167]:
test_data_pd1_rem1 = test_data_pd1.combine_first(test_data_rem1)
test_data_pd1_rem1.head()

,item,recoms
0,28759795,"{'31413158': 96.0, '31594180': 88.0, '28759792..."
1,32712593,"{'32712245': 112.0, '32712635': 88.0, '3271264..."
2,29616882,"{'29596054': 227.0, '29596018': 221.0, '295960..."
3,6242189,NaN
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [168]:
test_data_pd1_rem1.recoms.count()

60312

## Popular items can complement pairs

In [113]:
# Group predictions by item and create arrays (recoms, weight)
out_pop = pairs_test_pop.groupBy(col('test_item').name('item')) \
                                 .agg(collect_list(array('recoms', 'weight')).alias('recoms'))
%time out_pop.show(5)

+--------+--------------------+
|    item|              recoms|
+--------+--------------------+
|13078902|[[32698107, 0.0],...|
| 1513794|[[32867052, 0.0],...|
|15637425|[[3865066, 4.8076...|
| 1574856|[[3865066, 4.8076...|
|18555582|[[32538709, 0.0],...|
+--------+--------------------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 17.1 s


In [114]:
%time res_pop = out_pop.toPandas()
res_pop.head(2)

CPU times: user 1.93 s, sys: 524 ms, total: 2.45 s
Wall time: 7.56 s


,item,recoms
0,13078902,"[[5371570, 4.818181818181818], [28656389, 4.44..."
1,1513794,"[[33170854, 0.0], [6247843, 4.636363636363637]..."


In [115]:
# Make dict from recoms, convert weights to int and select top100 by weight
%time res_pop['recoms'] = res_pop['recoms'].apply(dict) \
            .apply(lambda x: dict((k, float(v)) for k,v in x.items())) \
            .apply(lambda x: dict(heapq.nlargest(100, x.items(), key=lambda i: i[1])))
res_pop.head(2)

CPU times: user 8.68 s, sys: 400 ms, total: 9.08 s
Wall time: 9.04 s


,item,recoms
0,13078902,"{'32973320': 5.0, '33300866': 5.0, '31012909':..."
1,1513794,"{'5559317': 5.0, '30868477': 5.0, '32973320': ..."


### Add pop items for quick submission after finding common friends and catalog pairs (weights = Rating))

In [116]:
# Make sure we have a correct order of items
test_data_pd_pop = test_data_pd.merge(res_pop, on='item', how='left')
test_data_pd_pop.head()

,item,recoms
0,28759795,"{'33300839': 5.0, '28906612': 5.0, '32562739':..."
1,32712593,"{'32105352': 5.0, '33300839': 5.0, '33300866':..."
2,29616882,"{'31012909': 5.0, '25151943': 5.0, '32105352':..."
3,6242189,"{'32105352': 5.0, '30868477': 5.0, '5559317': ..."
4,32663967,"{'28906612': 5.0, '31012909': 5.0, '30868477':..."


In [144]:
test_data_pd1_rem1_pop = test_data_pd1_rem1.combine_first(test_data_pd_pop)
test_data_pd1_rem1_pop.head()

,item,recoms
0,28759795,"{'31413158': 96.0, '31594180': 88.0, '28759792..."
1,32712593,"{'32712245': 112.0, '32712585': 96.0, '3271263..."
2,29616882,"{'29596054': 227.0, '29596037': 200.0, '295961..."
3,6242189,"{'32105352': 5.0, '30868477': 5.0, '5559317': ..."
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [145]:
test_data_pd1_rem1_pop.recoms.count()

60956

### Save to submission file (optional)

In [146]:
# Write predictions to file
%time output = test_data_pd1_rem1_pop.to_dict(orient='records')
with open('../project02.txt', 'w') as f:
    for i in output:
        f.write(json.dumps(i) + '\n')

CPU times: user 1.18 s, sys: 0 ns, total: 1.18 s
Wall time: 1.19 s


## !!! Result from friends and catalogs - 0.2285 (2019-06-25 17:52) => Further experiments (not finished) - search for more pairs from path_data and parent_id to improve the result

## Idea 3: Join test items with common path items

### Join path data with catalog data to get itemid

In [153]:
%time path.count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 11.1 s


54504

In [169]:
path_items = path.join(catalog_data, on='catalogid', how='left_outer') \
                 .filter('itemid is not null')
%time path_items.show(5)

+---------+--------+--------+--------+--------------------+--------+
|catalogid|path1_id|path2_id|path3_id|            path_txt|  itemid|
+---------+--------+--------+--------+--------------------+--------+
|  1139010| 1139001| 1137932| 1137924| Словари    Испан...|26881367|
|  1140991| 1140889| 1140879| 1132527| Исторические ром...|31587713|
|  1134295| 1134293| 1132524|    null|Женская обувь Обу...|17425282|
|  1138615| 1138564| 1138364| 1138260|Общие работы по в...| 6907117|
|  1144454| 1139735| 1139704| 1139697| Другие иностранн...| 7868912|
+---------+--------+--------+--------+--------------------+--------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.11 s


In [158]:
%time path_items.filter('itemid is not null').count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 8.32 s


10382286

### Join the remaining paired test items with path data

In [171]:
pairs_test_p = pairs_test_rem1.join(catalog_data.select(col('itemid').name('test_item'), 'catalogid'), \
                                    on='test_item', how='left_outer') \
                              .join(path, on='catalogid', how='left_outer') \
                              .dropDuplicates() \
                              .filter('path1_id is not null')
%time pairs_test_p.show(5)

+---------+---------+--------+------+--------+--------+--------+--------------------+
|catalogid|test_item|  recoms|weight|path1_id|path2_id|path3_id|            path_txt|
+---------+---------+--------+------+--------+--------+--------+--------------------+
|  1140682|  5471998| 2390668|   5.0| 1140575| 1139698| 1139697|Литературоведение...|
|  1178128| 31696989| 7288533|   4.5| 1159332| 1159330| 1145489|Леска, плетеный ш...|
|  1139655|  5197692|18089356|   5.0| 1139615| 1139545| 1137925|Напитки и коктейл...|
|  1134295| 30605019|31782898|   5.0| 1134293| 1132524|    null|Женская обувь Обу...|
|  1139633|  1574808| 3758115|   4.5| 1139545| 1137925| 1137924|Мясо. Птица. Пель...|
+---------+---------+--------+------+--------+--------+--------+--------------------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 5.4 s


In [172]:
pairs_test_p.rdd.getNumPartitions()

200

In [173]:
pairs_test_p = pairs_test_p.coalesce(20).cache()

In [194]:
%time pairs_test_p.count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 725 ms


719901

#### Note: some of rem1 items have been filtered out with path1=null !!!

In [198]:
%time pairs_test_p.select('test_item').distinct().count()

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 1.74 s


12387

### Join remaining test items with path items to find pairs in same path1

In [177]:
pairs_test_p1 = pairs_test_p.select('test_item', 'path1_id').dropDuplicates() \
                    .join(path_items.select('path1_id', 'itemid'), on='path1_id', how='left_outer') \
                    .dropDuplicates() \
                    .join(rating_data, on='itemid', how='left_outer') \
                    .filter('rating is not null') \
                    .withColumn('rating', col('rating')/2)
%time pairs_test_p1.show(5)

+--------+--------+---------+------+
|  itemid|path1_id|test_item|rating|
+--------+--------+---------+------+
|19968199| 1082047| 32364892|   2.0|
| 7382372| 1082047| 32364892|   2.5|
| 5594059| 1082047| 32364892|   2.5|
|13645326| 1082047| 32364892|   2.5|
|21386832| 1082047| 32364892|   2.5|
+--------+--------+---------+------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 4.34 s


In [178]:
pairs_test_p1.rdd.getNumPartitions()

200

In [184]:
pairs_test_p1 = pairs_test_p1.coalesce(8).cache()

In [179]:
# remove duplicates with items found by catalogid
pairs_test_p1 = pairs_test_p1.join(pairs_test_p.select('test_item', col('recoms').name('itemid')), \
                                   on=['test_item', 'itemid'], how='left_anti')
%time pairs_test_p1.show(5)

+---------+--------+--------+------+
|test_item|  itemid|path1_id|rating|
+---------+--------+--------+------+
|  1027248| 4289401| 1139449|   2.5|
|  1027248| 4541564| 1139449|   1.5|
|  1049388| 6842077| 1138616|   2.5|
|  1049388| 8791488| 1138616|   2.5|
| 10620657|24838404| 1160069|   1.5|
+---------+--------+--------+------+
only showing top 5 rows



In [185]:
%time pairs_test_p1.count()

CPU times: user 452 ms, sys: 144 ms, total: 596 ms
Wall time: 56min 17s


6801916

In [201]:
# Union remaining pairs and new recoms from path1
pairs_test_p11 = pairs_test_rem1.union(pairs_test_p1.select('test_item', \
                                                            col('itemid').name('recoms'), \
                                                            col('rating').name('weight')))
%time pairs_test_p11.show(5)

+---------+--------+-----------------+
|test_item|  recoms|           weight|
+---------+--------+-----------------+
| 18555582|19944594|              5.0|
| 18555582| 5432548|4.333333492279053|
| 18555582|20181397|              3.5|
| 18555582| 7628405|              3.0|
| 18555582|20181398|              4.0|
+---------+--------+-----------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 397 ms


In [204]:
%time pairs_test_p11.select('test_item').distinct().count()

CPU times: user 0 ns, sys: 4 ms, total: 4 ms
Wall time: 1.61 s


13434

In [203]:
# Number of test items with num recoms>100
%time pairs_test_p11.groupBy('test_item').agg(collect_list('recoms').name('recoms')) \
                .withColumn('num_recoms', size('recoms')).filter('num_recoms >= 100').count()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.99 s


9813

### Collect path recoms into array

In [205]:
# Group predictions by item and create arrays (recoms, weight)
out_p1 = pairs_test_p11.withColumnRenamed('test_item', 'item') \
                    .groupBy('item').agg(collect_list(array('recoms', 'weight')).alias('recoms'))
%time out_p1.show(5)

+--------+--------------------+
|    item|              recoms|
+--------+--------------------+
|18555582|[[26185096, 2.25]...|
|19148459|[[29810969, 2.5],...|
|19148526|[[5532376, 1.5], ...|
|19467872|[[7330063, 1.5], ...|
|19705179|[[27865407, 2.5],...|
+--------+--------------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 3.87 s


### Save path recoms to Pandas

In [206]:
%time res_p1 = out_p1.toPandas()
res_p1.head(2)

CPU times: user 3.6 s, sys: 812 ms, total: 4.41 s
Wall time: 14.7 s


,item,recoms
0,18555582,"[[27796721, 2.5], [24055618, 2.5], [26185099, ..."
1,19148459,"[[24708765, 2.5], [26418904, 2.5], [29810991, ..."


### Transform path recoms same way as batch1

In [207]:
# Make dict from recoms, convert weights from string to float and select top100 by weight
%time res_p1['recoms'] = res_p1['recoms'].apply(dict) \
            .apply(lambda x: dict((k, float(v)) for k,v in x.items())) \
            .apply(lambda x: dict(heapq.nlargest(100, x.items(), key=lambda i: i[1])))
res_p1.head(2)

CPU times: user 17 s, sys: 1.95 s, total: 19 s
Wall time: 18.9 s


,item,recoms
0,18555582,"{'19944594': 5.0, '7518810': 5.0, '3820346': 5..."
1,19148459,"{'24839584': 128.0, '19967396': 69.0, '3187959..."


### Add path records to test_data_pd for submission

In [208]:
# Make sure we have a correct order of items
test_data_p1 = test_data_pd.merge(res_p1, on='item', how='left')
test_data_p1.head()

,item,recoms
0,28759795,"{'28759817': 208.0, '31413158': 96.0, '3159418..."
1,32712593,"{'32712245': 112.0, '32712585': 96.0, '3271263..."
2,29616882,NaN
3,6242189,NaN
4,32663967,NaN


In [209]:
test_data_pd1_p1 = test_data_pd1.combine_first(test_data_p1)
test_data_pd1_p1.head()

,item,recoms
0,28759795,"{'28759817': 208.0, '31413158': 96.0, '3159418..."
1,32712593,"{'32712245': 112.0, '32712585': 96.0, '3271263..."
2,29616882,"{'29596054': 227.0, '29596018': 221.0, '295960..."
3,6242189,NaN
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [210]:
test_data_pd1_p1.recoms.count()

60312

### Add pop items for correct number of rows (or add tail below before pop!!!)

In [239]:
test_data_pd1_p1_pop = test_data_pd1_p1.combine_first(test_data_pd_pop)
test_data_pd1_p1_pop.head()

,item,recoms
0,28759795,"{'28759817': 208.0, '31413158': 96.0, '3159418..."
1,32712593,"{'32712245': 112.0, '32712585': 96.0, '3271263..."
2,29616882,"{'29596054': 227.0, '29596018': 221.0, '295960..."
3,6242189,"{'32105352': 5.0, '30868477': 5.0, '5559317': ..."
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [240]:
test_data_pd1_p1_pop.recoms.count()

60956

### Save path recoms to submission file (optional)

In [241]:
# Write predictions to file
%time output = test_data_pd1_p1_pop.to_dict(orient='records')
with open('../project02.txt', 'w') as f:
    for i in output:
        f.write(json.dumps(i) + '\n')

CPU times: user 1.16 s, sys: 68 ms, total: 1.22 s
Wall time: 1.22 s


## Alternatively: Find recoms for the "tail" of test items without any pairs with Friends or Catalogs

In [215]:
tail = test_data.select(col('item').name('test_item')) \
                .join(pairs_test_after_c.filter('recoms is not null').select('test_item').distinct(), \
                      on='test_item', how='left_anti') \
                .join(catalog_data.select('catalogid', col('itemid').name('test_item')), \
                                     on='test_item', how='left_outer')
%time tail.show(5)

+---------+---------+
|test_item|catalogid|
+---------+---------+
| 30306599|     null|
| 31484003|  1174273|
| 32502762|     null|
| 20088256|     null|
| 29673776|     null|
+---------+---------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 4.73 s


In [216]:
tail.rdd.getNumPartitions()

200

In [217]:
tail = tail.coalesce(4).cache()

In [218]:
%time tail.count()

CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 1min


587

In [225]:
tail = tail.filter('catalogid is not null').dropDuplicates()
%time tail.show(5)

+---------+---------+
|test_item|catalogid|
+---------+---------+
|  5358822|  1153581|
| 32386324|  1174735|
| 22436824|  1166437|
| 18925203|  1177689|
| 27063931|  1140536|
+---------+---------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 572 ms


In [226]:
tail.count()

71

In [227]:
tail = tail.join(catalog_data.select('catalogid', col('itemid').name('recoms')), on='catalogid', how='left_outer')
%time tail.show(5)

+---------+---------+--------+
|catalogid|test_item|  recoms|
+---------+---------+--------+
|  1156654| 18095974| 5290553|
|  1156654| 18095974|18095967|
|  1156654| 18095974|18095974|
|  1156654| 18095974|27260953|
|  1156654| 18095974|18095960|
+---------+---------+--------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.52 s


In [228]:
tail = tail.join(rating_data.select(col('itemid').name('recoms'), 'rating'), \
                 on='recoms', how='left_outer') \
           .fillna(0, subset=['rating']) \
           .withColumnRenamed('rating', 'weight')
%time tail.show(5)

+--------+---------+---------+------+
|  recoms|catalogid|test_item|weight|
+--------+---------+---------+------+
| 5290553|  1156654| 18095974|   0.0|
|18095960|  1156654| 18095974|   0.0|
|18095972|  1156654| 18095974|   0.0|
|27260953|  1156654| 18095974|   0.0|
|18095967|  1156654| 18095974|   0.0|
+--------+---------+---------+------+
only showing top 5 rows

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 3.1 s


In [229]:
# Group predictions by item and create arrays (recoms, weight)
out_tail = tail.withColumnRenamed('test_item', 'item') \
               .groupBy('item').agg(collect_list(array('recoms', 'weight')).alias('recoms'))
%time out_tail.show(5)

+--------+--------------------+
|    item|              recoms|
+--------+--------------------+
|31484003|[[31708478, 0.0],...|
|31578022|[[32027681, 0.0],...|
|25104693|[[25105425, 0.0],...|
|18095974|[[18095960, 0.0],...|
|29357014|[[29355373, 0.0],...|
+--------+--------------------+
only showing top 5 rows

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.2 s


In [230]:
%time res_tail = out_tail.toPandas()
res_tail.head(2)

CPU times: user 40 ms, sys: 8 ms, total: 48 ms
Wall time: 7.71 s


,item,recoms
0,31484003,"[[30937525, 0.0], [31216045, 0.0], [31778913, ..."
1,31578022,"[[33664288, 0.0], [31709341, 0.0], [32804179, ..."


In [231]:
# Make dict from recoms, convert weights from string to float and select top100 by weight
%time res_tail['recoms'] = res_tail['recoms'].apply(dict) \
            .apply(lambda x: dict((k, float(v)) for k,v in x.items())) \
            .apply(lambda x: dict(heapq.nlargest(100, x.items(), key=lambda i: i[1])))
res_tail.head(2)

CPU times: user 136 ms, sys: 0 ns, total: 136 ms
Wall time: 132 ms


,item,recoms
0,31484003,"{'30937525': 0.0, '31216045': 0.0, '31778913':..."
1,31578022,"{'33664288': 0.0, '31709341': 0.0, '32804179':..."


In [232]:
# Make sure we have a correct order of items
test_data_pd_tail = test_data_pd.merge(res_tail, on='item', how='left')
test_data_pd_tail.head()

,item,recoms
0,28759795,NaN
1,32712593,NaN
2,29616882,NaN
3,6242189,NaN
4,32663967,NaN


In [233]:
test_data_pd1_p1_tail = test_data_pd1_p1.combine_first(test_data_pd_tail)
test_data_pd1_p1_tail.head()

,item,recoms
0,28759795,"{'28759817': 208.0, '31413158': 96.0, '3159418..."
1,32712593,"{'32712245': 112.0, '32712585': 96.0, '3271263..."
2,29616882,"{'29596054': 227.0, '29596018': 221.0, '295960..."
3,6242189,NaN
4,32663967,"{'32663968': 56.0, '32106864': 40.0, '32486753..."


In [234]:
test_data_pd1_p1_tail.recoms.count()

60379

In [236]:
test_data_pd1_p1_tail_pop = test_data_pd1_p1_tail.combine_first(test_data_pd_pop)

In [237]:
test_data_pd1_p1_tail_pop.recoms.count()

60956

In [238]:
# Write predictions to file
%time output = test_data_pd1_p1_tail_pop.to_dict(orient='records')
with open('../project02.txt', 'w') as f:
    for i in output:
        f.write(json.dumps(i) + '\n')

CPU times: user 1.2 s, sys: 68 ms, total: 1.27 s
Wall time: 1.27 s


# Stop Spark

In [242]:
sc.stop()